In [1]:
import numpy as np
import torch

In [2]:
a = torch.ones(3)
print(a)
print(a[1])
print(a[1].item())
print(float(a[1]))

tensor([1., 1., 1.])
tensor(1.)
1.0
1.0


In [3]:
points = torch.tensor([4.0, 1.0, 5.0, 3.0, 2.0, 1.0])
print(points)

points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
print(points)
print(points.shape)
print(points[1])

points = torch.zeros(3,2)
print(points)

tensor([4., 1., 5., 3., 2., 1.])
tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])
torch.Size([3, 2])
tensor([5., 3.])
tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])


In [4]:
points = torch.zeros(3,2)
print(points.shape)
print(points[None].shape)
print(points.shape)

torch.Size([3, 2])
torch.Size([1, 3, 2])
torch.Size([3, 2])


In [5]:
# A single RGB image of size 5*5
# shape [channels, rows, columns]
img_t = torch.randn(3, 5, 5) 
# A batch of two RGB images
# shape [batch, channels, rows, columns]
batch_t = torch.randn(2, 3, 5, 5) 

# Average over the channel dimension
# .mean(-3): the mean value along the third-to-last dimension
img_gray_naive = img_t.mean(-3)
batch_gray_naive = batch_t.mean(-3)
print(img_t.shape, batch_t.shape)
print(img_gray_naive.shape, batch_gray_naive.shape)

torch.Size([3, 5, 5]) torch.Size([2, 3, 5, 5])
torch.Size([5, 5]) torch.Size([2, 5, 5])


In [6]:
img_t = torch.randn(3, 5, 5) 
batch_t = torch.randn(2, 3, 5, 5) 

## Weighted mean
# Create weights
weights = torch.tensor([0.2126, 0.7152, 0.0722])
# Add new trailing dimensions twice by .unsqueeze(-1), so that the tensor has shape [3,1,1]
unsqueezed_weights = weights.unsqueeze(-1).unsqueeze_(-1)
# Weighted mean using broadcasting
# During broadcasting, unsqueezed_weights's shape changes from [3,1,1] tp [3,5,5]
# which means every pixel gains the same weights for its 3 channels
img_weights = (img_t * unsqueezed_weights)
img_gray_weighted = img_weights.sum(-3)
# unsqueezed_weights's shape changes from [3,1,1] tp [2,3,5,5]
# which means every pixel in every image gains the same weights for its 3 channels
batch_weights = (batch_t * unsqueezed_weights)
batch_gray_weighted = batch_weights.sum(-3)

# Weighted average using einsum
img_gray_weighted_fancy = torch.einsum('...chw,c->...hw', img_t, weights)
batch_gray_weighted_fancy = torch.einsum('...chw,c->...hw', batch_t, weights)

In [7]:
img_t = torch.randn(3, 5, 5) 
batch_t = torch.randn(2, 3, 5, 5) 

# Named dimensions
weights_named = torch.tensor([0.2126, 0.7152, 0.0722], names=('channels',))
print(weights_named)
img_named = img_t.refine_names(..., 'channels', 'rows', 'columns')
print(img_named.shape, img_named.names)
batch_named = batch_t.refine_names(..., 'channels', 'rows', 'columns')
print(batch_named.shape, batch_named.names)

# Add dimensions by aligning
weights_aligned = weights_named.align_as(img_named)
print(weights_aligned.shape, weights_aligned.names)

# Sum over a named dimension
gray_named = (img_named * weights_aligned).sum('channels')
print(gray_named.shape, gray_named.names)

# Unname a tensor
gray_plain = gray_named.rename(None)
print(gray_plain.shape, gray_plain.names)

tensor([0.2126, 0.7152, 0.0722], names=('channels',))
torch.Size([3, 5, 5]) ('channels', 'rows', 'columns')
torch.Size([2, 3, 5, 5]) (None, 'channels', 'rows', 'columns')
torch.Size([3, 1, 1]) ('channels', 'rows', 'columns')
torch.Size([5, 5]) ('rows', 'columns')
torch.Size([5, 5]) (None, None)


C:\Users\bai_f\AppData\Local\Temp\ipykernel_24996\1882677003.py:5: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at C:\cb\pytorch_1000000000000\work\c10/core/TensorImpl.h:1931.)
  weights_named = torch.tensor([0.2126, 0.7152, 0.0722], names=('channels',))


In [9]:
# Specifying the numeric type with dtype
# torch.double == torch.float64
double_points = torch.ones(10, 2, dtype=torch.double)
print(double_points.dtype)
# dtype=torch.short == torch.int16
short_points = torch.tensor([[1, 2], [3, 4]], dtype=torch.short)
print(short_points.dtype)

# Default type
# Default float type: torch.float64
double_points = torch.ones(10, 2)
print(double_points.dtype)
# Default in type: torch.int64
short_points = torch.tensor([[1, 2], [3, 4]])
print(short_points.dtype)

torch.float64
torch.int16
torch.float32
torch.int64


In [11]:
# Change dtypes
double_points = torch.zeros(10, 2).double()
print(double_points.dtype)
short_points = torch.ones(10, 2).short()
print(short_points.dtype)

# Another way
double_points = torch.zeros(10, 2).to(torch.double)
print(double_points.dtype)
short_points = torch.ones(10, 2).to(dtype=torch.short)
print(short_points.dtype)

torch.float64
torch.int16
torch.float64
torch.int16


In [19]:
# View and storage
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
# The code in the book, deprecated in the current pytorch version
print( points.storage() )

# Recommended version
print( points.untyped_storage() )

# index into a storage manually
points_storage = points.storage()
print( points_storage[0] )

 4.0
 1.0
 5.0
 3.0
 2.0
 1.0
[torch.storage.TypedStorage(dtype=torch.float32, device=cpu) of size 6]
 0
 0
 128
 64
 0
 0
 128
 63
 0
 0
 160
 64
 0
 0
 64
 64
 0
 0
 0
 64
 0
 0
 128
 63
[torch.storage.UntypedStorage(device=cpu) of size 24]
4.0


In [23]:
# In-place operations
# These operations are marked by a trailing underscore in their name, 
# like zero_, which indicates that the method operates in place 
# by modifying the input instead of creating a new output tensor
a = torch.ones(3, 2)
a.zero_()
print(a)

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])
tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])
tensor([[1., 1.],
        [1., 1.],
        [1., 1.]])


In [25]:
# Any method without the trailing underscore leaves the source tensor unchanged and
# instead returns a new tensor
a = torch.tensor([1,2,3])
print( a.acosh() )
print(a)

tensor([0.0000, 1.3170, 1.7627])
tensor([1, 2, 3])


In [47]:
# Offset
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
print( points )
second_point = points[1]
print( second_point ) 
print( points.storage() )
# The offset of second_point is 2, since it is the 2nd row of 'points'.
# The two elements in the 1st row are skipped, resulting in an offset 2
print( second_point.storage_offset() )

# The first row starts at the beginning of the storage, so the offset is 0
print( points[0].storage_offset() )


# Stride
# a stride contains the number of steps needed to increase the index by 1 along each dimension,
# starting from the 1st element. 
# In this case, in order to move from a row to the next row, we need to take 2 steps
# To move from a column to another column (along a row), we only need to take 1 step
print(points.stride())

# Accessing an element i, j in a 2D tensor results in accessing 
# the storage_offset + stride[0] * i + stride[1] * j element in the storage.
i,j = 2,1
print(points[i,j])
ps = points.storage()
offset = points.storage_offset()
stride = points.stride()
print( ps[offset + stride[0] * i + stride[1] * j ]  )

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])
tensor([5., 3.])
 4.0
 1.0
 5.0
 3.0
 2.0
 1.0
[torch.storage.TypedStorage(dtype=torch.float32, device=cpu) of size 6]
2
0
(2, 1)
tensor(1.)
1.0


In [41]:
# Operations on a subtensor may change the original tensor
# So sometimes we clone the subtensor (make a copy)
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
second_point = points[1].clone()
second_point[0] = 10.0
print(second_point)
print(points)

tensor([10.,  3.])
tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])


In [45]:
# Transposition
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
print(points)
# t() is a shorthand for transpose(), only for 2D tensors
print(points.t()) 
print(points.transpose(0,1))
# Sharing the memory
print( id(points.t()) == id(points.transpose(0,1)) )

# Change in stride
points_t = points.t()
print(points.stride())
print(points_t.stride())

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])
tensor([[4., 5., 2.],
        [1., 3., 1.]])
tensor([[4., 5., 2.],
        [1., 3., 1.]])
True
(2, 1)
(1, 2)
